# Guide for the compilation of the INCA Core <br>implementation of a simple hydrological model

This document is meant to be a hands-on guide for the compilation of the INCA Core which is at its essence a model-building tool, written in C++, that allows for rapid model prototyping at the cost of computational performance. It is meant to be deployed as a _library_ against which your program should _link_ in order to create models.

Before going through this page, the reader is encouraged to browse the existing technical manual and the _*\.pptx_ presentations describing the Core, its philosophy and implementation. These can be found on [github repository](https://github.com/biogeochemistry/INCA/tree/master/documentation) where the INCA source is stored. Please see the [preliminary requirements](#prel_req) section of this document for an explanation on how to get access to the repository.

In short, the Core is a **model-building tool** resulting from the necessity of providing a structured, clear, and easily maintainable code base with which to build  and further advance the INCA family of models. This necessity arose from 20+ years of near-organic growth of the INCA models and the limitations this imposed on maintainability.

It is written in C++ and the programming style is object oriented. One of the main arguments in favor of object-oriented programming is its ability to provide _encapsulation_, i.e. divide the code into layers of complexity. This requires careful planning for the structure of the code (and as such complicates structural changes), but allows rapid development, which is a definite benefit for model prototyping. Encapsulating data accesses and more importantly, operations on data allows to layering the degree of understanding of the C++ code required to interact with the core. 

At its outermost, or simplest, level, the Core allows for the creation of models by specifying model parameters, inputs, outputs and equations (and their linkages) and provides a variety of solvers (which can be specified and/or added). It automatically creates all looping traditionally required in time-dependent models.

In order to interact with the Core at this level it is not necessary to be fluent with C++ but familiarity with it is an advantage. This document deals with usage of the Core at this level and aims to:

* show how get the source code on your local machine.
* provide a guide on how to compile the Core.
* show how to create and run simple model using the Core.


<b>Python settings and imports</b>

You can safely ignore this section. It is only meant to group the **imports** for the _Python_ sections of this document. 

* Importing modules required to run this notebook
* Defining functions used in this notebook
* Setting plotting options

In [1]:
import pygments
from pygments import highlight
from pygments.lexers import MakefileLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import display,HTML
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib as plt

<a id="prel_req"></a>

## Preliminary setup

### Getting the source code 

- Get a [github](https://github.com/) account.
- Get access to the repository, in case you don't already have it. Access is granted by either [JLG](http://www.niva.no/en/se-ansatt?navn=Jose-Luis%20Guerrero%20Calidonio) or [RMC](http://www.niva.no/en/se-ansatt?navn=Raoul-Marie%20Couture).
- Clone the repository https://github.com/biogeochemistry/INCA.git to your computer. There are many ways of achieving this and you might already have your preferences. If you don't, you could:
    - in Windows: use the [github desktop](https://desktop.github.com/) and [follow the instructions therein](https://help.github.com/desktop/guides/contributing/cloning-a-repository-from-github-to-github-desktop/).
    - in _\*nix_: a shell should suffice. Allergies to command line interfaces can be cured with, amongst many options, something like [git cola](http://git-cola.github.io/). 
- Be it through shell or GUI, it boils down to feeding github the following command and providing your credentials when prompted:

<section>
<div class=warn>
**shell: ** git clone https://<span></span>github.com/biogeochemistry/INCA.git
<div/>
</section>

You should now have the source code, as well as the documentation in your local machine. 

The Core depends on two external libraries in order to run. Binaries are provided but they can also be compiled from source. This is explained in more detail in the [dependencies](#dependencies) section.

**The directory where the Core was cloned is called CORE_DIR from now on.**
<a id="dependencies"></a>

### Dependencies

#### Boost

<a id="boost"></a>

The [Boost C++ libraries](http://www.boost.org/) are a highly-regarded, peer-reviewed  collection of libraries, some of them of such high-quality and usefulness that they end up becoming part of the C++ standard.

Most Boost libraries are header-only, i.e. need only be placed in a _PATH_ accessible to your compiler (there are several ways of achieving this) but others need to be _linked_ and as such require prior compilation themselves. 

The Core requires the [boost::date_time](http://www.boost.org/doc/libs/1_63_0/doc/html/date_time.html) library which therefore needs to be obtained or compiled from source beforehand.

For windows, [binaries](https://sourceforge.net/projects/boost/files/boost-binaries/1.63.0/) can be obtained requiring no compilation.

Instructions to compile the Boost libraries can be found [here](http://www.boost.org/doc/libs/1_63_0/more/getting_started/windows.html) for Windows and [here](http://www.boost.org/doc/libs/1_63_0/more/getting_started/unix-variants.html) for _\*nix_.

A general piece of advice would be to use the same compiler throughout. This goes a long way to prevent linking problems. 

The Boost libraries are meant to be backwards compatible but, to put it bluntly, shit happens. The Core has been tested with versions 1.59.0 through 1.63.0. 

<a id="TinyXML2"></a>
#### TinyXML2

[TinyXML2](http://www.grinninglizard.com/tinyxml2/) is a lightweight XML parser that is used within the Core to read input files and/or generate output, although the latter is not recommended as it is not an efficient way of generating output.

It is available for download from [github](https://github.com/leethomason/tinyxml2).

#### A quick discussion on linking 

If your code is to make use of a library, it has to _link_ (point) to it. This linkage can be either _static_ or _dynamic_. With static linking the library is included within every copy (executable) of your compiled program whereas with dynamic linking the library is placed in a folder specified in the PATH _environment variable_ that your program has therefore access to.

In Windows you can [modify the PATH following this procedure.](http://superuser.com/questions/949560/how-do-i-set-system-environment-variables-in-windows-10)

In _\*nix_ you can:
<section>
   <div class="warn">
   **shell: ** echo $PATH
   <div/>
</section>

to see which folders are already included (so you place your library in one of them) or modify it to add a path of your convenience:

<section>
   <div class="warn">
   **shell: ** PATH=/your/path:$PATH
   <div/>
</section>

Static linking makes distribution and installation of the executable simpler, since all is contained with a single package. Dynamic linking makes compilation simpler, as a change in a library would only require a re-compilation of that library but not of the entire source. 

*The instructions given here are for static linking.*

### Compilers

A compiler converts code into machine language that your computer can understand. When it comes to C++ compilers, most follow the C++ standard, which is a set of instructions on how the code is to be converted to machine language. The standard is constantly evolving and might in some cases be subject to _interpretation_. 

Furthermore, different compilers follow different _philosophies_ on how code should or can be transformed into _assembly_ (the language your processor understands). Some have their own ideosyncrasies and might include extensions of the standard.

As such, combining libraries borne of different compilers, even different _versions_ of the same compiler is often a losing proposition or, if the stars align, a serious challenge to your sanity. 

The _standard_ that is followed by the code in the Core is called C++11. Depending on your compiler, this might be the default, needs to be activated with a _flag_, might be altogether unavailable or only partially implemented.

For the purpose of this document the compilers that were tested are:
- in Windows: [MinGW-w64 (GCC v5.3.0)](https://mingw-w64.org/doku.php)
- in _\*nix_: [GCC-5.4.0](https://gcc.gnu.org/)

Building MinGW or GCC from source (and configuring them correctly) is not for the faint-hearted but self-installing executables or debian packages are convenient to use. If you are not using a debian-friendly linux distro, google is the order of the day. 

**The Core relies on pragmas (inline definition of functions), a new feature in C++11. Therefore your compiler MUST have support for this standard enabled. GCC-5.4 does so by passing the flag -std=c++11 to the compiler. For more recent compilers, the flag might not be necessary as this is the default**


## Compiling the Core

- Windows users should note that the steps described in this guide have been implemented on Lucid Dream and development can be done there.

- For <i>*nix</i>, everything is setup in the Stallo supercomputer. Otherwise you need to _apt-get_ gcc-5.4.0 and perhaps set *update\_alternatives* in case you have different versions of gcc on your machine.
</b>
</pre>
h2:{counter-reset}

###  Installing and testing

-Windows: To get MingGW, you can start by following the installation guide provided [here](http://www.msys2.org) to get a shell that works for Windows. The _.exe_ file with the __x86_64__ extension should work for a reasonably current PC. You should now a have _msys2_ installed. The default installation directory is: _C:/msys64_ and that is were you go to start your shell.

- Start shell to install gcc: C:/msys64/msys2.exe
- Get the latest version of gcc and  of make:
<section>
<div class="warn">
**shell: ** pacman -Syuu gcc <br/>
**shell: ** pacman -Syuu make
<div/>
</section>
To verify the installation was succesfull run:
<section>
<div class="warn">
**shell: ** gcc -v
<div/>
</section>
- Add the following, **in the specified order** to the path environment variable:
    - <i>C:\msys64\usr\lib\gcc\x86\_64-pc-msys</i> 
    - _C:\msys64\usr\bin_    
- Compile a test program to check things are working. Save the following as a _hello.cpp_ somewhere in your system:
    <pre>
    #include&lt;iostream&gt;
    int main(int argc, char** argv){
    std::cout << "Hello world!" << std::endl;
    return 0;
    }
    </pre>
Open a command line (_Start/cmd_) and navigate to the directory where _hello.cpp_ was saved and compile using:
    <pre>
    g++ hello.cpp -p hello
    </pre>
Upon successfull compilation, _hello.exe_ will have been created. Run it and be greeted.

    

-Linux: 
<section>
   <div class="warn">
   **shell: ** sudo add-apt-repository ppa:ubuntu-toolchain-r/test <br/>
   **shell: ** sudo apt-get update <br/>
   **shell: ** sudo apt-get install gcc-5 g++-5
   <div/>
</section>

### Obtaining required libraries

You should have static versions of the [boost::date_time](#boost) and [TinyXML2](#TinyXML2) libraries available. A static library filename should look like _libraryName.a_ as opposed to _libraryName.dll_ (Windows) or _libraryName.so_ (_\*nix_).

#### TinyXML2 from source

First of all, [download the source from github](https://github.com/leethomason/tinyxml2).

- Windows: Open command line  and navigate to the location of source code, i.e. the folder called _tinyxml2-master_. Then:
    <section>
   <div class="warn">
   **cmd:** make
   <div/>
</section>
and run _xmltest.exe_ to check everything went fine. _libtinyxml.a_ should have now been created.

- __\*nix__: Same as Windows, but use your shell of preference.

You will need to copy _libtinyxml2.a_ and _libtinyxml2.h_ within a certain location in the Core directory tree. Alternatively, you need to make this folder accessible to the compiler. More on that to come.

#### Boost from source

The Core makes us of the _boost::date\_time_ library. Existing extensions to the Core, i.ev the GLUE application require other boost libraries so this guide explains how to compile all the boost libraries. This is a somewhat lengthy compilation but it can be run in the background.

Download the [boost source code](https://sourceforge.net/projects/boost/files/boost/1.63.0/) and extract the _.zip_ (or tarball) file to a location within your _C:/msys64/_ directory tree. For example:
_C:/msys64/usr/local/_

- Windows: open an **msys terminal (NOT a command line)**, _C:/msys64/msys2.exe_ in the boost root folder (*C:/msys64/usr/local/boost\_1\_63\_0*).
<section>
   <div class="warn">
   **msys2:** ./bootstrap.sh
   <div/>
</section>
and the boost.build engine (b2.exe) should appear in your directory. Still within the msys2 terminal:
<section>
   <div class="warn">
   **msys2:** ./b2 --build-dir=build toolset=gcc variant=release link=static threading=multi stage 
   <div/>
</section>

- __\*.nix__: same as for Windows, but with a shell of your choice.

This might take a while and will build all boost libraries. Please note that only static, release and multithreaded versions of the libraries are being built. This is sufficient for current purposes, but you might want to build debug versions if you intend to do development, e.g.
Similarly to _tinyxml2.a_ the path to the boost libraries needs to be given to the compiler. More on that to come.

*libboost\_date\_time.a* will have been created in *C:/msys64/usr/local/boost\_1\_63\_0/stage/lib*

### Compilation through IDE or makefile

There are many of ways of flogging a cat and many ways of building the Core. 

You can either use an IDE or command line tools like make, amongst many many options. They are all ways of telling your compiler where the things it needs are located and in which order the sources should be compiled (simplifying a lot). An IDE provides the convenience of a GUI for this.

An IDE project setup is provided for Codeblocks within the Core source. In order to make the default setup work
you need to place both _libtinyxml.a_ and *libboost\_date\_time.a* in particular folder within specific locations of the Core source tree as well. You need to place _tinyxml2.h_ and all boost header files (*C:/msys64/usr/local/boost\_1\_63\_0/boost*) within specific places in the source tree, all within the _libraries_ folder. Navigate within this subfolder and follow the _"Place here"_ signs.

Using an IDE in a remote environment (such as in a remote computer) is a bit of a pain and in this case command line compilation is an advantage. Therefore a makefile was created to build the core:

In [2]:
fid = open('makefileCore','r')
code = fid.read()
fid.close
result = highlight(code, MakefileLexer(),HtmlFormatter(hl_lines=list(range(17,30))))
display(HTML(result))

The above makefile is on the github repository for this notebook and is called _makefileCore_. It should seamleassly work if the folders where you placed the source for the Core and the required dependencies are the sames as the ones highlighted above. Otherwise you might need to either place the files in the highlighted paths or modify the makefile.

The highlight under the _Macros_ highlighed section defines the name of the folder where the compiled Core library will be saved within the Core source tree. You can also modify this to your liking, but is is a good idea to preserve as much information as you can about the build. For instance, if you are doing this in Linux you should change CND_PLATFORM to Linux. An IDE will do these changes for you automatically but here it is your own responsability.

Once the paths have been set or the libraries placed in the right location, either copy/paste the above or download _makefileCore_ from this notebook's repository, open an _msys2_ terminal and navigate to the makefile's directory and:

<section>
   <div class="warn">
   **msys2:** make -f makeFileCore 
   <div/>
</section>

Note that if you copy/pasted you need to save under the name _makefileCore_ **with no extensions**. Also, if you name the file as _makefile_ then
<section>
   <div class="warn">
   **msys2:** make 
   <div/>
</section>

will be sufficient.


After some wait, the Core will have been compiled as a static library _libdcore.a_ that will have been placed in the folder:

*CORE\_DIR/INCA/DCore/dist/Release/Windows*

We're now ready to start building models.

## Building a model with the Core

### The model

We are going to implement parts of the HBV model, starting with the snow routine. For a description of the model refer to the powerpoint below (that you can expand to fullscreen).

In [3]:
HTML('<iframe src="https://docs.google.com/presentation/d/1Vm-svJE-wpvF5IF3MgAVQYhZvnIS7XY7gjzYwlNRIPs/embed?start=false&loop=false&delayms=3000" frameborder="0" width="480" height="389" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>')

In [4]:
# Path to custom CSS style for this document

def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [5]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

<IPython.core.display.Javascript object>